# Precisely Geodata Exploration

Working with data available through the [Precisely Data Experience sample datasets](https://data.precisely.com/samples). Specifically, `School Boundaries USA` downloaded in Shape format.

Schema:
https://docs.google.com/spreadsheets/d/18CDcZxSHvGXVVMDstur1yrHyum8i4qA4/edit#gid=782773548
    
**NOTE**: You may use [nbviewer](https://nbviewer.org/) to view this notebook outside your local Jupyter environment.  
[Click here](https://nbviewer.org/github/PMattox/fs/blob/main/precisely_exploration.ipynb) 

In [1]:
# pip install geopandas

In [2]:
# pip install folium

In [3]:
# IMPORTS
import geopandas as gpd
import pandas as pd

import os
import urllib.request
import requests
import shutil
from pathlib import Path
from zipfile import ZipFile

import matplotlib.pyplot as plt
from matplotlib import pyplot

import folium

from shapely.geometry import Point, Polygon

## School Boundaries (Shapefile)

In [4]:
filepath = '/Users/preston.mattox@finalsite.com/Downloads/SCHOOL_BOUNDARIES_USA_202312_SHP_SAMPLE/data/'
file = 'school_attendance_zone_objects_usa_sample.shp'

In [5]:
saz = gpd.read_file(filepath+file)

# column names to lowercase
saz.columns= saz.columns.str.lower()

saz.head(3)

,obj_id,obj_name,obj_typ,obj_subtcd,obj_subtyp,country,metro,lat,lon,reldate,obj_area,geometry
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.585586,-111.821031,20231126,2885227.634,"POLYGON ((-111.82347 40.59148, -111.82293 40.5..."
1,16074985,SILVER MESA ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.587210,-111.844863,20231126,2926275.545,"POLYGON ((-111.83516 40.59361, -111.83488 40.5..."
2,16074987,MIDVALLEY ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.610288,-111.880360,20231126,4316524.948,"POLYGON ((-111.86597 40.62190, -111.86599 40.6..."


In [6]:
# examine datatypes within the dataset
print(saz.geom_type.unique())
print(saz.dtypes)

['Polygon' 'MultiPolygon']
obj_id           int64
obj_name        object
obj_typ         object
obj_subtcd      object
obj_subtyp      object
country         object
metro           object
lat            float64
lon            float64
reldate         object
obj_area       float64
geometry      geometry
dtype: object


In [7]:
# plot the data on a map via folium

# create a base map centered on Salt Lake City, UT
map_saz = folium.Map(
    location=[40.68, -111.89],
    zoom_start=10,
)

# set up the annotations
tooltip = folium.GeoJsonTooltip(
    fields=["obj_name"],
    aliases=["School:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# add the school zones on the map
folium.GeoJson(saz[['geometry','obj_name']], 
               style_function=lambda feature: {'fillColor': '#2196F3', 'color': '#2196F3'},
              tooltip=tooltip
              ).add_to(map_saz)

# display the map
map_saz

In [8]:
# plot a random subset for easier examination

# create a base map centered on Salt Lake City, UT
map_saz = folium.Map(
    location=[40.68, -111.89],
    zoom_start=10,
)

# set up the annotations
tooltip = folium.GeoJsonTooltip(
    fields=["obj_name"],
    aliases=["School:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# add the school zones on the map
folium.GeoJson(saz[['geometry','obj_name']].sample(5), 
               style_function=lambda feature: {'fillColor': '#2196F3', 'color': '#2196F3'},
              tooltip=tooltip
              ).add_to(map_saz)

# display the map
map_saz

## School Attributes 

We started with `school_attendance_zone_objects_usa_sample.shp`, which ONLY includes zone info - this is not linked to the corresponding schools proper.  
We need to grab `school_attendance_zone_tertiary_attributes_usa_sample.csv` to get basic info, then connect that to `school_point_primary_attributes_usa_sample.csv`. This will provide us with everything we need: zone geometry, school name, and grades served.  

In [9]:
# find the file for attendance zone attributes
file_saz_t = 'school_attendance_zone_tertiary_attributes_usa_sample.csv'
saz_t = pd.read_csv(filepath+file_saz_t)
saz_t.head()

,att_id,obj_id_saz,obj_id_pnt,low_grade,high_grade
0,52747150,16075723,15117447,09,12
1,52747281,16075361,15116930,PK,5
2,52747563,16075475,15117254,KG,6
3,52747753,16075521,15117308,KG,6
4,52748050,16075317,15117200,07,8


In [10]:
# join with the attributes data to get low/high grades
saz_all_att = saz.join(saz_t.set_index('obj_id_saz'), on='obj_id', how='inner')
saz_all_att.head(3)

,obj_id,obj_name,obj_typ,obj_subtcd,obj_subtyp,country,metro,lat,lon,reldate,obj_area,geometry,att_id,obj_id_pnt,low_grade,high_grade
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.585586,-111.821031,20231126,2885227.634,"POLYGON ((-111.82347 40.59148, -111.82293 40.5...",52785550,15116961,PK,5
1,16074985,SILVER MESA ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.587210,-111.844863,20231126,2926275.545,"POLYGON ((-111.83516 40.59361, -111.83488 40.5...",52755754,15116955,KG,5
2,16074987,MIDVALLEY ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.610288,-111.880360,20231126,4316524.948,"POLYGON ((-111.86597 40.62190, -111.86599 40.6...",52756111,15116944,KG,5


In [11]:
# now add data about the schools themselves (i.e. the "point data")
file_s_att = 'school_point_primary_attributes_usa_sample.csv'
s_att = pd.read_csv(filepath+file_s_att)
s_att.head()

,obj_id,nces_schid,nces_disid,grtschl_id,choice,coextensiv,students,teachers,pk,kg,...,eighth,ninth,tenth,eleventh,twelfth,low_grade,high_grade,gender,school_typ,ed_level
0,15116924,490014200286,4900142,4902028.0,False,False,581.0,26.00,False,True,...,False,False,False,False,False,KG,5,NaN,R,P
1,15116926,490014200288,4900142,4902030.0,False,False,227.0,17.00,False,True,...,False,False,False,False,False,KG,5,NaN,R,P
2,15116927,490014200291,4900142,4902031.0,False,False,2240.0,82.96,False,False,...,False,True,True,True,True,09,12,NaN,R,H
3,15116928,490014200292,4900142,4902032.0,False,False,418.0,18.50,False,True,...,False,False,False,False,False,KG,5,NaN,R,P
4,15116929,490014200293,4900142,4902033.0,False,False,896.0,39.25,False,False,...,True,False,False,False,False,06,8,NaN,R,M


In [12]:
# bring it all together into one mega df
s_att = s_att.rename(columns={"low_grade": "low_grade_att", "high_grade": "high_grade_att"}, errors="raise")

saz_all_att = saz_all_att.join(s_att.set_index('obj_id'), on='obj_id_pnt', how='inner')
saz_all_att

,obj_id,obj_name,obj_typ,obj_subtcd,obj_subtyp,country,metro,lat,lon,reldate,...,eighth,ninth,tenth,eleventh,twelfth,low_grade_att,high_grade_att,gender,school_typ,ed_level
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.585586,-111.821031,20231126,...,False,False,False,False,False,PK,5,NaN,R,P
1,16074985,SILVER MESA ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.587210,-111.844863,20231126,...,False,False,False,False,False,KG,5,NaN,R,P
2,16074987,MIDVALLEY ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.610288,-111.880360,20231126,...,False,False,False,False,False,KG,5,NaN,R,P
3,16075168,ELK RUN ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,None,40.676653,-112.144352,20231126,...,False,False,False,False,False,KG,6,NaN,R,PM
4,16075170,ACADEMY PARK ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.673413,-112.000957,20231126,...,False,False,False,False,False,KG,6,NaN,R,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,16360273,OLENE WALKER ELEMENTARY,School,SAZ,School Attendance Zone,USA,"Salt Lake City, UT",40.694575,-111.907703,20231126,...,False,False,False,False,False,KG,5,NaN,R,P
221,16360449,MOUNTAIN POINT ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,None,40.452802,-111.959201,20231126,...,False,False,False,False,False,KG,6,NaN,R,PM
222,16363218,ASPEN ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,None,40.531711,-112.115001,20231126,...,False,False,False,False,False,KG,6,NaN,R,PM
223,16363219,OAK LEAF ELEMENTARY SCHOOL,School,SAZ,School Attendance Zone,USA,None,40.506730,-112.077923,20231126,...,False,False,False,False,False,KG,6,NaN,R,PM


In [13]:
# what columns did we end up with?
saz_all_att.columns

Index(['obj_id', 'obj_name', 'obj_typ', 'obj_subtcd', 'obj_subtyp', 'country',
       'metro', 'lat', 'lon', 'reldate', 'obj_area', 'geometry', 'att_id',
       'obj_id_pnt', 'low_grade', 'high_grade', 'nces_schid', 'nces_disid',
       'grtschl_id', 'choice', 'coextensiv', 'students', 'teachers', 'pk',
       'kg', 'first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh',
       'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth', 'low_grade_att',
       'high_grade_att', 'gender', 'school_typ', 'ed_level'],
      dtype='object')

In [14]:
# focus in on a few columns that we care about
saz_all_att[['obj_id', 'obj_name','geometry','nces_schid', 
             'students', 'teachers', 'pk',  'low_grade', 'high_grade', 'low_grade_att','high_grade_att']]

,obj_id,obj_name,geometry,nces_schid,students,teachers,pk,low_grade,high_grade,low_grade_att,high_grade_att
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,"POLYGON ((-111.82347 40.59148, -111.82293 40.5...",490014200768,479.0,25.23,True,PK,5,PK,5
1,16074985,SILVER MESA ELEMENTARY SCHOOL,"POLYGON ((-111.83516 40.59361, -111.83488 40.5...",490014200658,570.0,25.48,False,KG,5,KG,5
2,16074987,MIDVALLEY ELEMENTARY SCHOOL,"POLYGON ((-111.86597 40.62190, -111.86599 40.6...",490014200313,496.0,23.50,False,KG,5,KG,5
3,16075168,ELK RUN ELEMENTARY SCHOOL,"POLYGON ((-112.11084 40.70057, -112.11012 40.7...",490036001412,645.0,26.31,False,KG,6,KG,6
4,16075170,ACADEMY PARK ELEMENTARY SCHOOL,"POLYGON ((-111.98659 40.67595, -111.98660 40.6...",490036000196,376.0,19.00,False,KG,6,KG,6
...,...,...,...,...,...,...,...,...,...,...,...
220,16360273,OLENE WALKER ELEMENTARY,"POLYGON ((-111.90699 40.70712, -111.90678 40.7...",4900360XXX33,0.0,NaN,False,KG,5,KG,5
221,16360449,MOUNTAIN POINT ELEMENTARY SCHOOL,"POLYGON ((-111.91356 40.48984, -111.91280 40.4...",490042001543,686.0,28.63,False,KG,6,KG,6
222,16363218,ASPEN ELEMENTARY SCHOOL,"POLYGON ((-112.03289 40.56597, -112.03198 40.5...",490042001568,501.0,21.95,False,KG,6,KG,6
223,16363219,OAK LEAF ELEMENTARY SCHOOL,"POLYGON ((-112.03734 40.52239, -112.03695 40.5...",4900420XXX01,0.0,NaN,False,KG,6,KG,6


In [15]:
# as a check, make sure all the low grades from both tables match
saz_all_att.apply(lambda row: all(i in row.low_grade_att for i in row.low_grade), axis=1).sort_values

<bound method Series.sort_values of 0      True
1      True
2      True
3      True
4      True
       ... 
220    True
221    True
222    True
223    True
224    True
Length: 225, dtype: bool>

In [16]:
# as a check, make sure all the HIGH grades from both tables match
saz_all_att.apply(lambda row: all(i in str(row.high_grade_att) for i in str(row.high_grade)), axis=1).sort_values

<bound method Series.sort_values of 0      True
1      True
2      True
3      True
4      True
       ... 
220    True
221    True
222    True
223    True
224    True
Length: 225, dtype: bool>

### External validation of this joined data

We can use the [NCES school search](https://nces.ed.gov/ccd/schoolsearch/) to spot check these. 

`490014200768` should resolve to [Quail Hollow Elementary](https://nces.ed.gov/ccd/schoolsearch/school_list.asp?Search=1&InstName=&SchoolID=490014200768&Address=&City=&State=&Zip=&Miles=&County=&PhoneAreaCode=&Phone=&DistrictName=&DistrictID=&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1), serving grades K-5.


In [17]:
# pull Quail Hollow Elementary's NCES ID into from our df
test1 = saz_all_att.loc[saz_all_att['nces_schid'].isin(['490014200768'])][[
            'obj_id', 'obj_name','nces_schid','low_grade','high_grade','geometry']]
test1

,obj_id,obj_name,nces_schid,low_grade,high_grade,geometry
0,16074983,QUAIL HOLLOW ELEMENTARY SCHOOL,490014200768,PK,5,"POLYGON ((-111.82347 40.59148, -111.82293 40.5..."


In [18]:
# map Quail Hollow Elementary
# create a base map centered on Salt Lake City, UT
map_test1 = folium.Map(
    location=[40.68, -111.89],
    zoom_start=10,
)

# set up the annotations
tooltip = folium.GeoJsonTooltip(
    fields=["obj_name", "low_grade", "high_grade"],
    aliases=["School:", "Low Grade:", "High Grade:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# add the school zones on the map
folium.GeoJson(test1, 
               style_function=lambda feature: {'fillColor': '#2196F3', 'color': '#2196F3'},
              tooltip=tooltip
              ).add_to(map_test1)

# find the bounds of this one poly and adjust the zoom to match
bounds = test1.total_bounds.tolist()
map_test1.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])

# display the map
map_test1

Most of these look good, but there are a few trouble spots. Not sure this matters to us, but it's worth calling out.

For example, [Olene Walker Elementary](https://nces.ed.gov/ccd/schoolsearch/school_detail.asp?Search=1&InstName=OLENE+WALKER&SchoolType=1&SchoolType=2&SchoolType=3&SchoolType=4&SpecificSchlTypes=all&IncGrade=-1&LoGrade=-1&HiGrade=-1&ID=490036000259), NCES School ID: `490036000259`, doesn't resolve correctly from the Precisely data.

In [19]:
saz_all_att.loc[saz_all_att['nces_schid'].isin(['490036000259'])]

,obj_id,obj_name,obj_typ,obj_subtcd,obj_subtyp,country,metro,lat,lon,reldate,...,eighth,ninth,tenth,eleventh,twelfth,low_grade_att,high_grade_att,gender,school_typ,ed_level


In [20]:
# ...but Olene Walker Elementary definiely is in our dataset
saz_all_att[saz_all_att['obj_name'].str.contains('OLENE', regex=True)][[
            'obj_id', 'obj_name','nces_schid','low_grade','high_grade','geometry']]

,obj_id,obj_name,nces_schid,low_grade,high_grade,geometry
220,16360273,OLENE WALKER ELEMENTARY,4900360XXX33,KG,5,"POLYGON ((-111.90699 40.70712, -111.90678 40.7..."


----

# Demo

As a demonstration of this data capability, grab a single random sample from the dataset and map it with supporting info.

In [21]:
df_demo = saz_all_att.sample(1)

# create a base map centered on Salt Lake City, UT
map_demo = folium.Map(
    location=[40.68, -111.89],
    zoom_start=10,
)

# set up the annotations
tooltip = folium.GeoJsonTooltip(
    fields=["obj_name", "low_grade", "high_grade"],
    aliases=["School:", "Low Grade:", "High Grade:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

# add the school zones on the map
folium.GeoJson(df_demo, 
               style_function=lambda feature: {'fillColor': '#2196F3', 'color': '#2196F3'},
              tooltip=tooltip
              ).add_to(map_demo)

# find the bounds of this one poly and adjust the zoom to match
bounds = df_demo.total_bounds.tolist()
map_demo.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])

# display the 
map
map_demo

----

**References:**  
https://help.precisely.com/r/School-Boundaries/Main/en-US/School-Boundaries-Product-Guide/Common-operations/Retrieving-the-NCES-ID

https://nces.ed.gov/ccd/schoolsearch/

https://python-visualization.github.io/folium/latest/user_guide/geojson/geojson_popup_and_tooltip.html
https://nbviewer.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true